### Dataset Description

#### Dataset Proposed by Kristina   Store Sales - Time Series Forecasting

https://kaggle.com/competitions/store-sales-time-series-forecasting

### Dataset Description


In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

#### File Descriptions and Data Field Information


#### train.csv


- The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
- store_nbr identifies the store at which the products are sold.
- family identifies the type of product sold.
- sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.


#### test.csv


- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.
 

#### stores.csv


- Store metadata, including city, state, type, and cluster.
- cluster is a grouping of similar stores.


#### oil.csv


-Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)


#### holidays_events.csv


- Holidays and Events, with metadata
- NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
- 
Additional Notes

- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [1]:
from pydoc import describe

import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
import os


In [3]:
path = './data_favorita/'
os.listdir(path)

data_oil=pd.read_csv(path+'oil.csv')
data_holidays_events=pd.read_csv(path+'holidays_events.csv')
data_stores=pd.read_csv(path+'stores.csv')
data_train=pd.read_csv(path+'train.csv',parse_dates=['date'])
#data_test=pd.read_csv(path+'test.csv',parse_dates=['date'])
data_transactions=pd.read_csv(path+'transactions.csv')


In [4]:
data_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [5]:
data_stores.describe([1/3,2/3])

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
33.3%,18.666667,6.000000
50%,27.500000,8.500000
66.7%,36.333333,11.000000
max,54.000000,17.000000


In [6]:
data_stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [7]:
data_stores.duplicated().sum()

np.int64(0)

In [8]:
data_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


#### No duplicates and null found so this looks fairly clean in the first go

#### Now lets look at the transactions data



In [9]:
data_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [10]:
data_transactions.duplicated().sum()

np.int64(0)

In [11]:
data_transactions.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [12]:
data_transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [13]:
data_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


##### let us change the dtype of the date column

In [14]:
try:
	data_transactions["date"] = pd.to_datetime(data_transactions["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_transactions["date"].describe()

It worked!


count                            83488
mean     2015-05-20 16:07:40.866232064
min                2013-01-01 00:00:00
25%                2014-03-27 00:00:00
50%                2015-06-08 00:00:00
75%                2016-07-14 06:00:00
max                2017-08-15 00:00:00
Name: date, dtype: object

#### Lets join the store and the transactions data


In [15]:
data_transactions_stores=data_transactions.merge(data_stores,on='store_nbr',how='left')

In [16]:
data_transactions_stores.head()

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
1,2013-01-02,1,2111,Quito,Pichincha,D,13
2,2013-01-02,2,2358,Quito,Pichincha,D,13
3,2013-01-02,3,3487,Quito,Pichincha,D,8
4,2013-01-02,4,1922,Quito,Pichincha,D,9


#### Now lets look at the holidays and events data

In [17]:
data_holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [18]:
try:
	data_holidays_events["date"] = pd.to_datetime(data_holidays_events["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_holidays_events["date"].describe()

It worked!


count                              350
mean     2015-04-24 00:45:15.428571392
min                2012-03-02 00:00:00
25%                2013-12-23 06:00:00
50%                2015-06-08 00:00:00
75%                2016-07-03 00:00:00
max                2017-12-26 00:00:00
Name: date, dtype: object

In [19]:
data_holidays_events.describe()

,date
count,350
mean,2015-04-24 00:45:15.428571392
min,2012-03-02 00:00:00
25%,2013-12-23 06:00:00
50%,2015-06-08 00:00:00
75%,2016-07-03 00:00:00
max,2017-12-26 00:00:00


In [20]:
data_holidays_events.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [21]:
data_holidays_events.duplicated().sum()

np.int64(0)

In [22]:
data_holidays_events.isna().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [23]:
data_holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


#### Now lets look at the oil data

In [24]:
data_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [25]:
data_oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [26]:
data_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

#### So oIl data has null data and will need cleaning or imputation.Lets analyze the oil file further


In [27]:
data_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


Let us change teh date to date time64 object type

In [28]:
try:
	data_oil["date"] = pd.to_datetime(data_oil["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_oil["date"].describe()


It worked!


count                   1218
mean     2015-05-02 12:00:00
min      2013-01-01 00:00:00
25%      2014-03-03 06:00:00
50%      2015-05-02 12:00:00
75%      2016-06-30 18:00:00
max      2017-08-31 00:00:00
Name: date, dtype: object

In [29]:
# Impute missing oil prices with the price from the previous row
#data_oil['dcoilwtico'].fillna(method='ffill', inplace=True)
data_oil["dcoilwtico"].ffill()

0         NaN
1       93.14
2       92.97
3       93.12
4       93.20
        ...  
1213    47.65
1214    46.40
1215    46.46
1216    45.96
1217    47.26
Name: dcoilwtico, Length: 1218, dtype: float64

In [30]:
data_oil.duplicated().sum()

np.int64(0)

In [31]:
data_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [32]:
mean_price = data_oil['dcoilwtico'].mean()
#data_oil['dcoilwtico'].fillna(mean_price, inplace=True)
data_oil.fillna({'dcoilwtico': mean_price}, inplace=True)


In [33]:
data_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

#### so oil data doesnt have any duplicate values and there are no more null values and the date is converted to datetime64 which will et us do mathmatical operations on it

#### Let us now look at the train data


In [34]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [35]:
data_train.describe()

,id,date,store_nbr,sales,onpromotion
count,3.000888e+06,3000888,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2015-04-24 08:27:04.703088384,2.750000e+01,3.577757e+02,2.602770e+00
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,2014-02-26 18:00:00,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2015-04-24 12:00:00,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,2016-06-19 06:00:00,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,2017-08-15 00:00:00,5.400000e+01,1.247170e+05,7.410000e+02
std,8.662819e+05,NaN,1.558579e+01,1.101998e+03,1.221888e+01


In [36]:
data_train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [37]:
data_train.duplicated().sum()

np.int64(0)

In [38]:
data_train.head(20)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0
7,7,2013-01-01,1,CLEANING,0.0,0
8,8,2013-01-01,1,DAIRY,0.0,0
9,9,2013-01-01,1,DELI,0.0,0


In [39]:
# Convert the 'family' column to integer codes
data_train['family'], _ = pd.factorize(data_train['family'])

# Verify the conversion
data_train.tail(100)

,id,date,store_nbr,family,sales,onpromotion
3000788,3000788,2017-08-15,6,32,47.575,0
3000789,3000789,2017-08-15,7,0,5.000,0
3000790,3000790,2017-08-15,7,1,0.000,0
3000791,3000791,2017-08-15,7,2,11.000,2
3000792,3000792,2017-08-15,7,3,3999.000,9
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,28,438.133,0
3000884,3000884,2017-08-15,9,29,154.553,1
3000885,3000885,2017-08-15,9,30,2419.729,148
3000886,3000886,2017-08-15,9,31,121.000,8


In [40]:
df1 = data_train.merge(data_stores,  
                      on='store_nbr',
                      how='left')

In [41]:
df2 = df1.merge(data_transactions,
                on=['store_nbr', 'date'],
                how='left')
df3 = df2.merge(data_oil,
                on='date',
                how='left')


In [42]:
#date,type,locale,locale_name,description,transferred
# Based on the locale column value is "National" then do df3.merge(data_holidays_events) left on 'date' mark put holiday as 1 else 0
# Based on the locale column value is "local" and city name matches then then do df3.merge(data_holidays_events) left on 'date' mark put holiday as 1 else 0
# Based on the locale column value is "Regional" and State name matches then then do df3.merge(data_holidays_events) left on 'date' mark put holiday as 1 else 0

# Merge holidays data with df3
final_df = df3.merge(data_holidays_events, on='date', how='left')

# Initialize the is_holiday column with 0
final_df['is_holiday'] = 0

# Apply conditions for National holidays
final_df.loc[final_df['locale'] == 'National', 'is_holiday'] = 1

# Apply conditions for Local holidays (city matches)
final_df.loc[(final_df['locale'] == 'Local') &
              (final_df['city'] == final_df['locale_name']), 'is_holiday'] = 1

# Apply conditions for Regional holidays (state matches)
final_df.loc[(final_df['locale'] == 'Regional') &
              (final_df['state'] == final_df['locale_name']), 'is_holiday'] = 1






In [43]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 18 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        int64         
 4   sales         float64       
 5   onpromotion   int64         
 6   city          object        
 7   state         object        
 8   type_x        object        
 9   cluster       int64         
 10  transactions  float64       
 11  dcoilwtico    float64       
 12  type_y        object        
 13  locale        object        
 14  locale_name   object        
 15  description   object        
 16  transferred   object        
 17  is_holiday    int64         
dtypes: datetime64[ns](1), float64(3), int64(6), object(8)
memory usage: 419.5+ MB


In [44]:
# let us check for missing sales data and impute if missing by substituting the value with the previous days sales and transaction
# Check for missing sales and transaction data
# Run the imputation process 5 times
for i in range(5):
    print(f"Run {i+1}:")

    # Check for missing sales and transaction data
    print("Missing sales data before imputation:", final_df['sales'].isnull().sum())
    print("Missing transactions data before imputation:", final_df['transactions'].isnull().sum())

    # Impute missing sales and transactions with the previous day's values
    final_df['sales'] = final_df['sales'].ffill()
    final_df['transactions'] = final_df['transactions'].ffill()


    # Verify if missing values are handled
    print("Missing sales data after imputation:", final_df['sales'].isnull().sum())
    print("Missing transactions data after imputation:", final_df['transactions'].isnull().sum())
    print("-" * 50)

Run 1:
Missing sales data before imputation: 0
Missing transactions data before imputation: 249117
Missing sales data after imputation: 0
Missing transactions data after imputation: 561
--------------------------------------------------
Run 2:
Missing sales data before imputation: 0
Missing transactions data before imputation: 561
Missing sales data after imputation: 0
Missing transactions data after imputation: 561
--------------------------------------------------
Run 3:
Missing sales data before imputation: 0
Missing transactions data before imputation: 561
Missing sales data after imputation: 0
Missing transactions data after imputation: 561
--------------------------------------------------
Run 4:
Missing sales data before imputation: 0
Missing transactions data before imputation: 561
Missing sales data after imputation: 0
Missing transactions data after imputation: 561
--------------------------------------------------
Run 5:
Missing sales data before imputation: 0
Missing transa

In [45]:
import pandas as pd

# Make sure your DataFrame has at least: date, store_nbr, is_holiday
# final_df = pd.read_csv(...)  # Your real data

# Step 1: Convert date to datetime
final_df['date'] = pd.to_datetime(final_df['date'])
final_df = final_df.dropna(subset=['date']).reset_index(drop=True)

# Step 2: Add temporal features
final_df['day_of_week'] = final_df['date'].dt.dayofweek
final_df['is_weekend'] = final_df['day_of_week'].isin([5, 6]).astype(int)
final_df['month'] = final_df['date'].dt.month
final_df['week_of_year'] = final_df['date'].dt.isocalendar().week

final_df['week_of_year'] = final_df['week_of_year'].astype('int32')

# Step 3: Sort data
final_df = final_df.sort_values(['store_nbr', 'date']).reset_index(drop=True)



In [46]:
import numpy as np
import pandas as pd

def add_days_until_and_past_last_holiday(df):
    df = df.copy()

    # Ensure date is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Sort by store and date
    df.sort_values(['store_nbr', 'date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Create arrays to hold the calculated values
    days_until = np.full(len(df), np.nan)
    days_past_last = np.full(len(df), np.nan)

    # Process each store separately
    current_pos = 0
    for _, group in df.groupby('store_nbr', sort=False):
        group_len = len(group)
        dates = group['date'].values
        is_holiday = group['is_holiday'].values.astype(bool)

        # Filter holidays for this store
        holiday_dates = dates[is_holiday]

        # Calculate 'days_until_holiday' and 'days_past_last_holiday' for each row in the group
        if len(holiday_dates) > 0:
            # Calculate 'days_until_holiday' (Next holiday)
            delta_days_until = np.full(group_len, np.nan)
            for i in range(group_len):
                future_holidays = holiday_dates[holiday_dates > dates[i]]

                if future_holidays.size > 0:
                    delta_days_until[i] = (future_holidays.min() - dates[i]).astype('timedelta64[D]').astype(float)

            days_until[current_pos:current_pos + group_len] = delta_days_until

            # Calculate 'days_past_last_holiday' (Previous holiday)
            delta_days_past = np.full(group_len, np.nan)
            for i in range(group_len):
                past_holidays = holiday_dates[holiday_dates < dates[i]]

                if past_holidays.size > 0:
                    delta_days_past[i] = (dates[i] - past_holidays.max()).astype('timedelta64[D]').astype(float)

            days_past_last[current_pos:current_pos + group_len] = delta_days_past

        current_pos += group_len

    # Assign the results to the dataframe
    df['days_until_holiday'] = days_until
    df['days_past_last_holiday'] = days_past_last

    # Fill NaN values in both columns with the value 23
    df['days_until_holiday'].fillna(23, inplace=True)
    df['days_past_last_holiday'].fillna(10, inplace=True)

    # Return the updated dataframe
    return df




In [47]:
#final_df_sample = final_df

final_df = add_days_until_and_past_last_holiday(final_df)
#final_df = add_days_until_and_past_last_holiday(final_df)

#final_df_sample.describe()
final_df.head(20)

/var/folders/bm/9gklnwns2kxcypy3sp68vlc80000gn/T/ipykernel_26856/4260091425.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['days_until_holiday'].fillna(23, inplace=True)
/var/folders/bm/9gklnwns2kxcypy3sp68vlc80000gn/T/ipykernel_26856/4260091425.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,locale_name,description,transferred,is_holiday,day_of_week,is_weekend,month,week_of_year,days_until_holiday,days_past_last_holiday
0,0,2013-01-01,1,0,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
1,1,2013-01-01,1,1,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
2,2,2013-01-01,1,2,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
3,3,2013-01-01,1,3,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
4,4,2013-01-01,1,4,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
5,5,2013-01-01,1,5,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
6,6,2013-01-01,1,6,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
7,7,2013-01-01,1,7,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
8,8,2013-01-01,1,8,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0
9,9,2013-01-01,1,9,0.0,0,Quito,Pichincha,D,13,...,Ecuador,Primer dia del ano,False,1,1,0,1,1,4.0,10.0


In [48]:
#backup_final_df = final_df.copy()
#final_df = backup_final_df.copy()

In [49]:
final_df.describe()

,id,date,store_nbr,family,sales,onpromotion,cluster,transactions,dcoilwtico,is_holiday,day_of_week,is_weekend,month,week_of_year,days_until_holiday,days_past_last_holiday
count,3.054348e+06,3054348,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.053787e+06,2.175822e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06
mean,1.504277e+06,2015-04-26 12:11:45.717619968,2.750000e+01,1.600000e+01,3.590209e+02,2.617480e+00,8.481481e+00,1.689590e+03,6.800526e+01,9.019404e-02,3.008168e+00,2.876313e-01,6.222870e+00,2.530863e+01,2.086567e+01,2.079324e+01
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,2.619000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,7.546768e+05,2014-03-01 00:00:00,1.400000e+01,8.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,1.055000e+03,4.670000e+01,0.000000e+00,1.000000e+00,0.000000e+00,3.000000e+00,1.300000e+01,6.000000e+00,6.000000e+00
50%,1.507572e+06,2015-04-28 12:00:00,2.750000e+01,1.600000e+01,1.100000e+01,0.000000e+00,8.500000e+00,1.419000e+03,5.448000e+01,0.000000e+00,3.000000e+00,0.000000e+00,6.000000e+00,2.500000e+01,1.600000e+01,1.600000e+01
75%,2.255120e+06,2016-06-22 00:00:00,4.100000e+01,2.400000e+01,1.960110e+02,0.000000e+00,1.300000e+01,2.035000e+03,9.549000e+01,0.000000e+00,5.000000e+00,1.000000e+00,9.000000e+00,3.800000e+01,3.200000e+01,3.200000e+01
max,3.000887e+06,2017-08-15 00:00:00,5.400000e+01,3.200000e+01,1.247170e+05,7.410000e+02,1.700000e+01,8.359000e+03,1.106200e+02,1.000000e+00,6.000000e+00,1.000000e+00,1.200000e+01,5.300000e+01,7.800000e+01,7.800000e+01
std,8.662610e+05,NaN,1.558579e+01,9.521906e+00,1.107286e+03,1.225494e+01,4.649735e+00,9.383528e+02,2.523496e+01,2.864596e-01,2.001442e+00,4.526584e-01,3.374251e+00,1.475051e+01,1.815006e+01,1.815623e+01


In [50]:
# List of store numbers to exclude

'''
exclude_stores = [23]

# Filter out excluded stores
filtered_df = final_df[~final_df['store_nbr'].isin(exclude_stores)]

# Convert columns to numeric in case they're object types (for safety)
filtered_df['days_until_holiday'] = pd.to_numeric(filtered_df['days_until_holiday'], errors='coerce')
filtered_df['days_past_last_holiday'] = pd.to_numeric(filtered_df['days_past_last_holiday'], errors='coerce')

# Filter rows where EITHER column is NOT 23 and the values are DIFFERENT
filtered_df = filtered_df[
    ((filtered_df['days_until_holiday'] != 23) | (filtered_df['days_past_last_holiday'] != 23)) &
    (filtered_df['days_until_holiday'] != filtered_df['days_past_last_holiday'])
]

# Show the first 100 rows
print(filtered_df[['store_nbr', 'date', 'is_holiday', 'days_until_holiday', 'days_past_last_holiday']].head(100))


'''



"\nexclude_stores = [23]\n\n# Filter out excluded stores\nfiltered_df = final_df[~final_df['store_nbr'].isin(exclude_stores)]\n\n# Convert columns to numeric in case they're object types (for safety)\nfiltered_df['days_until_holiday'] = pd.to_numeric(filtered_df['days_until_holiday'], errors='coerce')\nfiltered_df['days_past_last_holiday'] = pd.to_numeric(filtered_df['days_past_last_holiday'], errors='coerce')\n\n# Filter rows where EITHER column is NOT 23 and the values are DIFFERENT\nfiltered_df = filtered_df[\n    ((filtered_df['days_until_holiday'] != 23) | (filtered_df['days_past_last_holiday'] != 23)) &\n    (filtered_df['days_until_holiday'] != filtered_df['days_past_last_holiday'])\n]\n\n# Show the first 100 rows\nprint(filtered_df[['store_nbr', 'date', 'is_holiday', 'days_until_holiday', 'days_past_last_holiday']].head(100))\n\n\n"

In [51]:
# Create a dataframe with unique combinations of city, locale_name, and state
unique_city_locale_state_df = final_df[['city', 'locale_name', 'state']].dropna().drop_duplicates()

# Reset index for clean viewing
unique_city_locale_state_df = unique_city_locale_state_df.reset_index(drop=True)

# Show the result
print(unique_city_locale_state_df)


          city                     locale_name      state
0        Quito                         Ecuador  Pichincha
1        Quito                           Manta  Pichincha
2        Quito                        Cotopaxi  Pichincha
3        Quito                          Cuenca  Pichincha
4        Quito                        Libertad  Pichincha
..         ...                             ...        ...
523  El Carmen  Santo Domingo de los Tsachilas     Manabi
524  El Carmen                     Santa Elena     Manabi
525  El Carmen                           Quito     Manabi
526  El Carmen                            Loja     Manabi
527  El Carmen                         Salinas     Manabi

[528 rows x 3 columns]


In [52]:
# Check if any column has NaN values in the dataframe
nan_columns = unique_city_locale_state_df.isna().any()

# Display which columns have NaN values
print(nan_columns)

# Optionally, count how many rows in each column have NaN values
nan_counts = unique_city_locale_state_df.isna().sum()
print(f"Number of NaN values per column:\n{nan_counts}")


city           False
locale_name    False
state          False
dtype: bool
Number of NaN values per column:
city           0
locale_name    0
state          0
dtype: int64


In [53]:
#unique_city_locale_state_df['state'].nunique()
#check the number of unique city

#unique_city_locale_state_df['city'].nunique()
#unique_city_locale_state_df['locale_name'].nunique()


In [54]:
# impute missing values in final_df for state ,city and locale_name columns in that order
# Fill missing 'state' values using 'city' and 'locale_name'
final_df['state'] = final_df.apply(    lambda row: unique_city_locale_state_df.loc[
        (unique_city_locale_state_df['city'] == row['city']) &
        (unique_city_locale_state_df['locale_name'] == row['locale_name']), 'state'
    ].values[0] if pd.isna(row['state']) and not unique_city_locale_state_df.loc[
        (unique_city_locale_state_df['city'] == row['city']) &
        (unique_city_locale_state_df['locale_name'] == row['locale_name'])
    ].empty else row['state'],
    axis=1
)


In [55]:
final_df['state'].isnull().sum()

np.int64(0)

In [56]:
# print the row whole state is nulll
print(final_df[final_df['state'].isnull()])
# drop this row
final_df = final_df.dropna(subset=['state'])
print(final_df[final_df['state'].isnull()])

Empty DataFrame
Columns: [id, date, store_nbr, family, sales, onpromotion, city, state, type_x, cluster, transactions, dcoilwtico, type_y, locale, locale_name, description, transferred, is_holiday, day_of_week, is_weekend, month, week_of_year, days_until_holiday, days_past_last_holiday]
Index: []

[0 rows x 24 columns]
Empty DataFrame
Columns: [id, date, store_nbr, family, sales, onpromotion, city, state, type_x, cluster, transactions, dcoilwtico, type_y, locale, locale_name, description, transferred, is_holiday, day_of_week, is_weekend, month, week_of_year, days_until_holiday, days_past_last_holiday]
Index: []

[0 rows x 24 columns]


In [57]:
final_df['city'].isnull().sum()


np.int64(0)

In [58]:
final_df['locale_name'].isnull().sum()

np.int64(2551824)

In [59]:
#drop the locale_name column
final_df.drop(columns=['locale_name','locale','description','transferred'], inplace=True)


In [60]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 20 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   id                      int64         
 1   date                    datetime64[ns]
 2   store_nbr               int64         
 3   family                  int64         
 4   sales                   float64       
 5   onpromotion             int64         
 6   city                    object        
 7   state                   object        
 8   type_x                  object        
 9   cluster                 int64         
 10  transactions            float64       
 11  dcoilwtico              float64       
 12  type_y                  object        
 13  is_holiday              int64         
 14  day_of_week             int32         
 15  is_weekend              int64         
 16  month                   int32         
 17  week_of_year            int32         
 18  da

In [61]:
# List of suspected categorical columns
categorical_cols = ['store_nbr', 'family','cluster','city', 'state', 'type_x', 'type_y', ]

# Show unique values for each
for col in categorical_cols:
    if col in final_df.columns:
        print(f"\n--- {col} ({final_df[col].nunique()} unique values) ---")
        print(final_df[col].unique())



--- store_nbr (54 unique values) ---
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54]

--- family (33 unique values) ---
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32]

--- cluster (17 unique values) ---
[13  8  9  4  6 15  7  3 12 16  1 10  2  5 11 14 17]

--- city (22 unique values) ---
['Quito' 'Santo Domingo' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil' 'Salinas' 'Daule' 'Babahoyo'
 'Quevedo' 'Playas' 'Libertad' 'Cuenca' 'Loja' 'Machala' 'Esmeraldas'
 'Manta' 'El Carmen']

--- state (16 unique values) ---
['Pichincha' 'Santo Domingo de los Tsachilas' 'Cotopaxi' 'Chimborazo'
 'Imbabura' 'Bolivar' 'Pastaza' 'Tungurahua' 'Guayas' 'Santa Elena'
 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas' 'Manabi']

--- type_x (5 unique values) ---
['D' 'B' 'C' 'E' 'A']

--- type_y (6 unique 

In [62]:
# List of categorical columns
categorical_cols = ['store_nbr', 'family', 'cluster', 'city', 'state', 'type_x', 'type_y']

# Perform one-hot encoding on the categorical columns
final_df_encoded = pd.get_dummies(final_df, columns=categorical_cols, drop_first=True)

# View the resulting dataframe after one-hot encoding
print(final_df_encoded.head())


   id       date  sales  onpromotion  transactions  dcoilwtico  is_holiday  \
0   0 2013-01-01    0.0            0           NaN   67.714366           1   
1   1 2013-01-01    0.0            0           NaN   67.714366           1   
2   2 2013-01-01    0.0            0           NaN   67.714366           1   
3   3 2013-01-01    0.0            0           NaN   67.714366           1   
4   4 2013-01-01    0.0            0           NaN   67.714366           1   

   day_of_week  is_weekend  month  ...  state_Tungurahua  type_x_B  type_x_C  \
0            1           0      1  ...             False     False     False   
1            1           0      1  ...             False     False     False   
2            1           0      1  ...             False     False     False   
3            1           0      1  ...             False     False     False   
4            1           0      1  ...             False     False     False   

   type_x_D  type_x_E  type_y_Bridge  type_y_Event

In [63]:
final_df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Columns: 159 entries, id to type_y_Work Day
dtypes: bool(146), datetime64[ns](1), float64(5), int32(3), int64(4)
memory usage: 693.3 MB


In [64]:
pd.set_option('display.max_columns', None)
final_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Columns: 159 entries, id to type_y_Work Day
dtypes: bool(146), datetime64[ns](1), float64(5), int32(3), int64(4)
memory usage: 693.3 MB


In [65]:
final_df_encoded.info

<bound method DataFrame.info of               id       date    sales  onpromotion  transactions  dcoilwtico  \
0              0 2013-01-01    0.000            0           NaN   67.714366   
1              1 2013-01-01    0.000            0           NaN   67.714366   
2              2 2013-01-01    0.000            0           NaN   67.714366   
3              3 2013-01-01    0.000            0           NaN   67.714366   
4              4 2013-01-01    0.000            0           NaN   67.714366   
...          ...        ...      ...          ...           ...         ...   
3054343  3000751 2017-08-15   59.619            0         802.0   47.570000   
3054344  3000752 2017-08-15   94.000            0         802.0   47.570000   
3054345  3000753 2017-08-15  915.371           76         802.0   47.570000   
3054346  3000754 2017-08-15    0.000            0         802.0   47.570000   
3054347  3000755 2017-08-15    3.000            0         802.0   47.570000   

         is_holiday

In [66]:
final_df_encoded[final_df_encoded.select_dtypes(include='bool').columns] = final_df_encoded.select_dtypes(include='bool').astype('int64')
final_df_encoded[final_df_encoded.select_dtypes(include='int32').columns] = final_df_encoded.select_dtypes(include='int32').astype('int64')


In [67]:
final_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Columns: 159 entries, id to type_y_Work Day
dtypes: datetime64[ns](1), float64(5), int64(153)
memory usage: 3.6 GB


In [68]:
#final_df_encoded.drop(columns=['days_past_last_holiday'], inplace=True)